<a href="https://colab.research.google.com/github/pavanswaroopl/NLP/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
import pathlib
import torch
import numpy as np
!pip install evaluate
import evaluate
import gensim.downloader
import gensim.models.keyedvectors as word2vec
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM
import tensorflow as tf

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_text=[]
train_sentiment=[]
test_text=[]
test_sentiment=[]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#for sentiment in ["pos","neg"]:
#  for text_file in (pathlib.Path('/content/drive/MyDrive/NLP/train')/sentiment).iterdir():
#    train_text.append(text_file.read_text())
#    if sentiment=='pos':
#      train_sentiment.append(0)
#    else:
#      train_sentiment.append(1)

#print(train_text[0])

for sentiment in ["pos", "neg"]:
    path = pathlib.Path('/content/drive/MyDrive/NLP/train') / sentiment
    for text_file in path.iterdir():
        with text_file.open() as f:
            train_text.append(f.read())
            train_sentiment.append(0 if sentiment == "pos" else 1)

print(train_text[0])

#for sentiment in ["pos","neg"]:
#  for text_file in (pathlib.Path('/content/drive/MyDrive/NLP/test')/sentiment).iterdir():
#    test_text.append(text_file.read_text())
#    if sentiment=='pos':
#      test_sentiment.append(0)
#    else:
#      test_sentiment.append(1)

#print(test_text[0])

for sentiment in ["pos", "neg"]:
    path = pathlib.Path('/content/drive/MyDrive/NLP/test') / sentiment
    for text_file in path.iterdir():
        with text_file.open() as f:
            test_text.append(f.read())
            test_sentiment.append(0 if sentiment == "pos" else 1)

print(test_text[0])

train_tokenized = tokenizer(train_text, truncation=True, padding=True)
test_tokenized = tokenizer(test_text, truncation=True, padding=True)

I felt it necessary to respond to the comments posted on the front page of this film's page because some of it was slightly misinformative.<br /><br />Originally I posted quotes from the original poster, but I wasn't sure if it was proper given that this is the "comments" index and not a message board (though we used to use 'em that way back before IMDb added the film message boards) so I will edit this to make it unnecessary.<br /><br />Well, first of all you may not be aware of this, but Gene Kelly first became famous for playing "Pal Joey" on Broadway in the original production. When Vincente Minnelli decided to make a Gershwin "panorama" film, he wanted Kelly's character to be more sophisticated than the "goody two shoes" roles he had been playing in most his films (with the exception of "For Me and My Gal"). Alan Jay Lerner was instructed to construct a new story set in Paris based on the story of "Pal Joey". This gave Kelly a chance to play his famous role from Broadway even thou

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_tokenized, train_sentiment)
test_dataset = Dataset(test_tokenized, test_sentiment)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
#FineTuning Model
modelFineTuning = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
modelFineTuning.classifier = torch.nn.Linear(768, 2)
modelFineTuning.num_labels = 2

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)
trainer_Fine_Tuning = Trainer(model=modelFineTuning,args=training_args,train_dataset=train_dataset,eval_dataset=test_dataset,compute_metrics=compute_metrics)
trainer_Fine_Tuning.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

Epoch,Training Loss,Validation Loss,Accuracy
1,0.278100,0.245568,0.913400
2,0.167600,0.306309,0.929040
3,0.065900,0.369716,0.929440


TrainOutput(global_step=9387, training_loss=0.18176126038163337, metrics={'train_runtime': 4951.9381, 'train_samples_per_second': 15.164, 'train_steps_per_second': 1.896, 'total_flos': 9946976965079040.0, 'train_loss': 0.18176126038163337, 'epoch': 3.0})

In [6]:
#LinearProbing model
modelLinearProbing = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
modelLinearProbing.classifier = torch.nn.Linear(768, 2)
modelLinearProbing.num_labels = 2

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)
trainer_Linear_Probing = Trainer(model=modelLinearProbing,args=training_args,train_dataset=train_dataset,eval_dataset=test_dataset,compute_metrics=compute_metrics,)

for param in modelLinearProbing.base_model.parameters():
    param.requires_grad = False

trainer_Linear_Probing.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

Epoch,Training Loss,Validation Loss,Accuracy
1,0.364600,0.355363,0.844800
2,0.342600,0.334415,0.857960


Epoch,Training Loss,Validation Loss,Accuracy
1,0.364600,0.355363,0.844800
2,0.342600,0.334415,0.857960
3,0.361100,0.332943,0.859160


TrainOutput(global_step=9387, training_loss=0.37530816511896026, metrics={'train_runtime': 2593.081, 'train_samples_per_second': 28.958, 'train_steps_per_second': 3.62, 'total_flos': 9946976965079040.0, 'train_loss': 0.37530816511896026, 'epoch': 3.0})

In [7]:
wordtovecmodel=gensim.downloader.load('glove-twitter-200')
tokenize = Tokenizer(num_words=88583)
tokenize.fit_on_texts(train_text)
words_index=tokenize.word_index
train_x = tokenize.texts_to_sequences(train_text)
maxlen=200
train_x=tf.keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
tokenize.fit_on_texts(test_text)
test_x = tokenize.texts_to_sequences(test_text)
test_x=tf.keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
embedding_matrix = np.zeros((len(words_index) + 1, maxlen))
for word, i in words_index.items():
  if word in wordtovecmodel:
    embedding_vector = wordtovecmodel[word]
    embedding_matrix[i] = embedding_vector
embedding_layer = Embedding(input_dim=len(words_index) + 1,output_dim=maxlen,weights=[embedding_matrix],input_length=maxlen,trainable=False)
modelLSTM = Sequential()
modelLSTM.add(embedding_layer)
modelLSTM.add(LSTM(units=100))
modelLSTM.add(Dense(1, activation='sigmoid'))
modelLSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelLSTM.summary())
modelLSTM.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
modelLSTM.fit(train_x,np.array(train_sentiment), epochs=3,validation_data=(test_x, np.array(test_sentiment)))

[=================================================-] 99.9% 758.1/758.5MB downloaded
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 200)          17716600  
                                                                 
 lstm (LSTM)                 (None, 100)               120400    
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 17,837,101
Trainable params: 120,501
Non-trainable params: 17,716,600
_________________________________________________________________
None
Epoch 1/3
783/783 [==============================] - 19s 17ms/step - loss: 0.5153 - accuracy: 0.7526 - val_loss: 0.7641 - val_accuracy: 0.5556
Epoch 2/3
783/783 [==============================] - 13s 17ms/step - l

In [8]:
print("Fine tuned model with layers",)
print(trainer_Fine_Tuning.evaluate(eval_dataset=test_dataset))
print("*****************************")
print("Linear Probing model i.e. after freezing the layers",)
print(trainer_Linear_Probing.evaluate(eval_dataset=test_dataset))
print("*****************************")
#trainer_Fine_Tuning.save_model("Finetuning_1")
#trainer_Linear_Probing.save_model("Finetuning_2")
loss, accuracy = modelLSTM.evaluate(test_x,np.array(test_sentiment))

Fine tuned model with layers


{'eval_loss': 0.36971569061279297, 'eval_accuracy': 0.92944, 'eval_runtime': 421.7912, 'eval_samples_per_second': 59.271, 'eval_steps_per_second': 7.409, 'epoch': 3.0}
*****************************
Linear Probing model i.e. after freezing the layers


{'eval_loss': 0.33294305205345154, 'eval_accuracy': 0.85916, 'eval_runtime': 419.5476, 'eval_samples_per_second': 59.588, 'eval_steps_per_second': 7.448, 'epoch': 3.0}
*****************************
782/782 [==============================] - 4s 5ms/step - loss: 0.8995 - accuracy: 0.5580
